# Monad transformers

## Outline

* Incentive for the Monad transformers

* ...

In this lesson, we will learn about Monad transformers and how you can use it.

## Incentive for Monad transformers

When we learned about the State monad we showed how you can implement the Tic-Tac-Toe with using this monad. 

A curious student might ask how to implement this game with State monads by geting the input from the user instead of the random funtion.

Below is the code that implements our costum `StateIO` state monad that allows IO actions inside of it.

In [ ]:
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE FlexibleInstances #-} 

import Control.Monad.Trans
import Control.Monad (ap, liftM)
import Control.Monad.State.Class (MonadState(get, put))

newtype StateIO s a = StateIO {runStateIO :: s -> IO (a, s)}

instance Functor (StateIO s) where
    fmap = liftM

instance Applicative (StateIO s) where
    pure = return
    (<*>) = ap

instance Monad (StateIO s) where
    return a = StateIO $ \s -> return (a, s)
    n >>= k = StateIO $ \s -> do (a, s') <- runStateIO n s
                                 runStateIO (k a) s'

instance MonadIO (StateIO s) where
    liftIO io = StateIO $ \st -> do x <- io
                                    return (x, st)

instance MonadState s (StateIO s) where
    get = StateIO $ \st -> return (st,st)
    put st = StateIO $ const $ return ((),st)

We see that the `StateIO` type similar as the `State` type is defined just that it returns a `IO (a,s)` tuple instead of `(a,s)`.

Then we create the **Functor**, **Applicative** and **Monad** instances for the `StateIO s` type.

If we want to use a IO action in the State monad we need to define the `liftIO` function which is part of the `MonadIO` type class.

To make an instance of this class for our `StateIO s` monad we need to import the **Control.Monad.Trans** module.

The `liftIO` function takes in a IO action and computes a new state where it performs the IO actions to get the return parameter for the state.

In the end we need to make and instance of **MonadState** type class for our `StateIO s` type and define the `get` and `put` functions.

For this we need also to include the language pragmas *MultiParamTypeClasses* and *FlexibleInstances*.

The *FlexibleInstances* pragma allows definition of type class instances with arbitrary nested types in the instance head.

Now that we created the `StateIO` type let's re-write the fibonacci example and use a IO action inside the `StateIO` monad.

In [ ]:
type FibNums = [Int]

main :: IO ()
main = do
    putStrLn "How many fibonacci numbers do you want to see:"
    n <- (read <$> getLine) :: IO Int
    let initialState = [n,0,1]
    process initialState

process :: FibNums -> IO ()
process st = do
    (finished, newSt) <- runStateIO addFib st
    if finished 
    then do
      putStrLn "Fibonacci numbers are:"
      print $ tail newSt
    else process newSt

addFib :: StateIO FibNums Bool
addFib = do
    st <- get
    let newSt = st ++ [last st + last (init st)]
        finished = length (tail newSt) == head newSt
    liftIO $ putStrLn $ "Printing new state in StateIO monad: " ++ show (tail newSt)
    put newSt
    return finished

main

We see that in the `addFib` state monad we can now make a output to the terminal if we use the `liftIO` function.

## Recap

In this lesson we've discussed:

- the motivation for introducing Monad transformers

- ...